In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, Imputer
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats

In [2]:
#Read in Data
train = pd.read_csv('dataFiles/application_train.csv')
test = pd.read_csv('dataFiles/application_test.csv')
bureau_data = pd.read_csv('dataFiles/bureau.csv')
bureau_balance_data = pd.read_csv('dataFiles/bureau_balance.csv')
prev_app_data = pd.read_csv('dataFiles/previous_application.csv')
pos_cash_balance_data = pd.read_csv('dataFiles/POS_CASH_balance.csv')
installments_data = pd.read_csv('dataFiles/installments_payments.csv')
cc_data = pd.read_csv('dataFiles/credit_card_balance.csv')

In [3]:
cc_data_grouped = cc_data.select_dtypes(exclude='object').groupby('SK_ID_PREV', as_index = False).agg({'MONTHS_BALANCE':[sum, 'median', 'mean', min, max],
                                                                       'AMT_BALANCE':[sum, 'median', 'mean', min, max],
                                                                       'AMT_CREDIT_LIMIT_ACTUAL':[sum, 'median', 'mean', min, max],
                                                                       'AMT_DRAWINGS_ATM_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_DRAWINGS_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_DRAWINGS_OTHER_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_DRAWINGS_POS_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_INST_MIN_REGULARITY':[sum, 'median', 'mean', min, max],
                                                                       'AMT_PAYMENT_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_PAYMENT_TOTAL_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_RECEIVABLE_PRINCIPAL':[sum, 'median', 'mean', min, max],
                                                                       'AMT_RECIVABLE':[sum, 'median', 'mean', min, max],
                                                                       'AMT_TOTAL_RECEIVABLE':[sum, 'median', 'mean', min, max],
                                                                       'CNT_DRAWINGS_ATM_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'CNT_DRAWINGS_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'CNT_DRAWINGS_OTHER_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'CNT_DRAWINGS_POS_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'CNT_INSTALMENT_MATURE_CUM':[sum, 'median', 'mean', min, max],
                                                                       'SK_DPD':[sum, 'median', 'mean', min, max],
                                                                       'SK_DPD_DEF':[sum, 'median', 'mean', min, max]
                                                                      })

In [4]:
cc_data_grouped.columns = ['_CC_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in cc_data_grouped.columns]

In [5]:
prev_app_data.columns = [i + '_PRE' if i != 'SK_ID_PREV' and i != 'SK_ID_CURR' else i for i in prev_app_data.columns]

In [6]:
prev_app_data_merged = prev_app_data.merge(cc_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [7]:
installments_data_grouped = installments_data.groupby('SK_ID_PREV', as_index = False).agg({'NUM_INSTALMENT_VERSION':[sum, 'median', 'mean', min, max],
                                                              'NUM_INSTALMENT_NUMBER':[sum, 'median', 'mean', min, max],
                                                              'DAYS_INSTALMENT': [sum, 'median', 'mean', min, max],
                                                              'DAYS_ENTRY_PAYMENT': [sum, 'median', 'mean', min, max],
                                                              'AMT_INSTALMENT': [sum, 'median', 'mean', min, max],
                                                              'AMT_PAYMENT': [sum, 'median', 'mean', min, max]})

In [8]:
installments_data_grouped.columns = ['_INST_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in installments_data_grouped.columns]

In [9]:
prev_app_data_merged = prev_app_data_merged.merge(installments_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [10]:
pos_cash_balance_data = pd.get_dummies(pos_cash_balance_data, drop_first = True)

In [11]:
pos_cash_balance_data_grouped = pos_cash_balance_data.groupby('SK_ID_PREV', as_index=False).agg({'MONTHS_BALANCE':[sum, 'median', 'mean', min, max],
                                                                                                 'CNT_INSTALMENT':[sum, 'median', 'mean', min, max],
                                                                                                 'CNT_INSTALMENT_FUTURE':[sum, 'median', 'mean', min, max],
                                                                                                 'CNT_INSTALMENT_FUTURE':[sum, 'median', 'mean', min, max],
                                                                                                 'SK_DPD':[sum, 'median', 'mean', min, max],
                                                                                                 'SK_DPD_DEF':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Amortized debt':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Approved':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Completed':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Returned to the store':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Signed':[sum, 'median', 'mean', min, max],
                                                                                                })

In [12]:
pos_cash_balance_data_grouped.columns = ['_POS_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in pos_cash_balance_data_grouped.columns]

In [13]:
prev_app_data_merged = prev_app_data_merged.merge(pos_cash_balance_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [14]:
bureau_balance_data_grouped = pd.get_dummies(bureau_balance_data).groupby('SK_ID_BUREAU', as_index=False).agg({'MONTHS_BALANCE':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_0':[sum, 'median', 'mean', min, max],
                                                                                                               'STATUS_1':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_2':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_3':[sum, 'median', 'mean', min, max],
                                                                                                               'STATUS_3':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_4':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_5':[sum, 'median', 'mean', min, max],
                                                                                                               'STATUS_X':[sum, 'median', 'mean', min, max]})

In [15]:
bureau_balance_data_grouped.columns = ['_BBD_'.join(col) if col != ('SK_ID_BUREAU', '') else col[0] for col in bureau_balance_data_grouped.columns]

In [16]:
bureau_data = bureau_data.merge(bureau_balance_data_grouped, how = 'left', left_on = 'SK_ID_BUREAU', right_on = 'SK_ID_BUREAU')

In [17]:
# Joining Bureau Grouped Data with Train Data
bureau_data_grouped = bureau_data.groupby('SK_ID_CURR', as_index=False).agg({'DAYS_CREDIT':[sum, 'median', 'mean', min, max],
                                                                             'CREDIT_DAY_OVERDUE':[sum, 'median', 'mean', min, max],
                                                                             'DAYS_CREDIT_ENDDATE':[sum, 'median', 'mean', min, max],
                                                                             'DAYS_ENDDATE_FACT':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_MAX_OVERDUE':[sum, 'median', 'mean', min, max],
                                                                             'CNT_CREDIT_PROLONG':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM_DEBT':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM_LIMIT':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM_OVERDUE':[sum, 'median', 'mean', min, max],
                                                                             'DAYS_CREDIT_UPDATE':[sum, 'median', 'mean', min, max],
                                                                             'AMT_ANNUITY':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_median':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_min':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_max':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_median':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_min':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_max':[sum, 'median', 'mean', min, max]})

In [18]:
bureau_data_grouped.columns = ['_'.join(col) if col != ('SK_ID_CURR', '') else col[0] for col in bureau_data_grouped.columns]

In [130]:
train_dummies = pd.get_dummies(train[['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',\
'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FONDKAPREMONT_MODE','HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']]).fillna(0)

In [131]:
test_dummies = pd.get_dummies(test[['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',\
'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FONDKAPREMONT_MODE','HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']]).fillna(0)

In [140]:
train_one_hot = pd.concat([train.select_dtypes(exclude=['object']), train_dummies], axis = 1)

test_one_hot = pd.concat([test.select_dtypes(exclude=['object']), test_dummies], axis = 1)

In [143]:
train_merged = train_one_hot.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test_one_hot.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [144]:
prev_app_data_subset = prev_app_data_merged[prev_app_data_merged.select_dtypes(exclude=['object']).columns.tolist()].drop('SK_ID_PREV', axis = 1)

In [145]:
prev_app_data_grouped = prev_app_data_subset.groupby('SK_ID_CURR', as_index=False).agg(sum)

In [146]:
#prev_app_data_grouped.columns = ['_'.join(col) if col != ('SK_ID_CURR', '') else col[0] for col in prev_app_data_grouped.columns]
#prev_app_data_grouped = prev_app_data_grouped.reset_index()

In [147]:
train_merged = train_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [148]:
own_car_median = train_merged.OWN_CAR_AGE.median()

In [149]:
def own_car_missing(x):
    if x['FLAG_OWN_CAR'] == 'N':
        return 28
    elif x['FLAG_OWN_CAR'] == 'Y' and pd.isnull(x['OWN_CAR_AGE']):
        return own_car_median
    else:
        return x['OWN_CAR_AGE']

In [150]:
train_merged_subset = train_merged.dropna(thresh=len(train_merged) - 100000, axis = 1)

In [151]:
new_cols = train_merged_subset.columns.tolist()
new_cols.remove('TARGET')

In [152]:
len(train_merged_subset.columns)

409

In [153]:
col_corr = train_merged_subset.corr()['TARGET'].sort_values()

In [167]:
column_corr_subset = col_corr[(col_corr >= 0) | (col_corr < 0)].index.values.tolist()

In [168]:
column_corr_subset.remove('TARGET')
column_corr_subset.remove('CODE_GENDER_XNA')
column_corr_subset.remove('NAME_FAMILY_STATUS_Unknown')
column_corr_subset.remove('NAME_INCOME_TYPE_Maternity leave')

In [169]:
train_subset = train_merged_subset[column_corr_subset]

test_subset = test_merged[column_corr_subset]

In [170]:
imputer = Imputer(strategy = 'median')
imputer.fit(train_subset)
train_merged_imputed = pd.DataFrame(imputer.transform(train_subset), columns = train_subset.columns)
test_merged_imputed = pd.DataFrame(imputer.transform(test_subset), columns = train_subset.columns)

In [171]:
train_merged_imputed['DAYS_EMPLOYED_^2'] = train_merged_imputed['DAYS_EMPLOYED'] ** 2
train_merged_imputed['AMT_GOODS_PRICE_^2'] = train_merged_imputed['AMT_GOODS_PRICE'] ** 2
train_merged_imputed['DAYS_CREDIT_^2'] = train_merged_imputed['DAYS_CREDIT_mean'] ** 2
train_merged_imputed['DAYS_BIRTH_^2'] = train_merged_imputed['DAYS_BIRTH'] ** 2

test_merged_imputed['DAYS_EMPLOYED_^2'] = test_merged_imputed['DAYS_EMPLOYED'] ** 2
test_merged_imputed['AMT_GOODS_PRICE_^2'] = test_merged_imputed['AMT_GOODS_PRICE'] ** 2
test_merged_imputed['DAYS_CREDIT_^2'] = test_merged_imputed['DAYS_CREDIT_mean'] ** 2
test_merged_imputed['DAYS_BIRTH_^2'] = test_merged_imputed['DAYS_BIRTH'] ** 2

In [172]:
poly_transformer = PolynomialFeatures(degree = 1)
poly_transformer.fit(train_merged_imputed)
train_poly_features = poly_transformer.transform(train_merged_imputed)

In [173]:
train_subset_poly = pd.DataFrame(train_poly_features, columns = poly_transformer.get_feature_names(
    input_features = train_merged_imputed.columns.tolist()
))

In [174]:
test_poly_features = poly_transformer.transform(test_merged_imputed)

In [175]:
test_subset_poly = pd.DataFrame(test_poly_features, columns = poly_transformer.get_feature_names(input_features = test_merged_imputed.columns.tolist()))

In [176]:
scaler = MinMaxScaler(feature_range = (0, 1))

In [177]:
scaler.fit(train_subset_poly)
train_scaled = scaler.transform(train_subset_poly)
test_scaled = scaler.transform(test_subset_poly)

In [178]:
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(iterations=1000, learning_rate=0.1)

In [179]:
cat_model.fit(train_scaled, train.TARGET)

0:	learn: 0.5865010	total: 1.43s	remaining: 23m 53s
1:	learn: 0.5070860	total: 2.67s	remaining: 22m 11s
2:	learn: 0.4464212	total: 3.94s	remaining: 21m 49s
3:	learn: 0.4019467	total: 5.16s	remaining: 21m 23s
4:	learn: 0.3683708	total: 6.66s	remaining: 22m 4s
5:	learn: 0.3422541	total: 7.87s	remaining: 21m 44s
6:	learn: 0.3236969	total: 9.05s	remaining: 21m 23s
7:	learn: 0.3081119	total: 10.2s	remaining: 21m 8s
8:	learn: 0.2964074	total: 11.5s	remaining: 21m 1s
9:	learn: 0.2875591	total: 12.8s	remaining: 21m 4s
10:	learn: 0.2799501	total: 14s	remaining: 21m 2s
11:	learn: 0.2746317	total: 15.6s	remaining: 21m 25s
12:	learn: 0.2699493	total: 17.7s	remaining: 22m 24s
13:	learn: 0.2664584	total: 19.5s	remaining: 22m 55s
14:	learn: 0.2634075	total: 21.1s	remaining: 23m 5s
15:	learn: 0.2608290	total: 23.2s	remaining: 23m 47s
16:	learn: 0.2590734	total: 24.7s	remaining: 23m 48s
17:	learn: 0.2574919	total: 26.1s	remaining: 23m 43s
18:	learn: 0.2559918	total: 27.4s	remaining: 23m 36s
19:	learn: 

In [180]:
test_y_cat = pd.DataFrame(cat_model.predict_proba(test_scaled))
submission_cat = pd.concat([test.SK_ID_CURR, test_y_cat], axis=1).drop(0, axis = 1)
submission_cat.columns = ['SK_ID_CURR', 'Target']

In [181]:
#max_sub_cat = submission_cat.Target.max()
#min_sub_cat = submission_cat.Target.min()

In [182]:
#submission_cat['Target'] = submission_cat['Target'].apply(lambda x: (x - min_sub_cat) / (max_sub_cat - min_sub_cat))

In [183]:
submission_cat.to_csv('reduced_cat18.csv', index = False)